In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jane-street-market-prediction/example_sample_submission.csv
/kaggle/input/jane-street-market-prediction/features.csv
/kaggle/input/jane-street-market-prediction/example_test.csv
/kaggle/input/jane-street-market-prediction/train.csv
/kaggle/input/jane-street-market-prediction/janestreet/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jane-street-market-prediction/janestreet/__init__.py


In [3]:
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]

In [4]:
features = ['feature_{}'.format(i) for i in range(0,130)]

In [5]:
fill_mode = lambda col: col.fillna(col.mode()[0])
train = train.apply(fill_mode, axis=0)
train.isna().sum()

date           0
weight         0
resp_1         0
resp_2         0
resp_3         0
              ..
feature_126    0
feature_127    0
feature_128    0
feature_129    0
ts_id          0
Length: 138, dtype: int64

In [6]:
train['action'] = ((train.resp > 0) * 1).astype(int)

In [7]:
train = train.drop(['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4'], axis=1)

In [8]:
X_train = train.loc[train.date < 425].drop(['date', 'ts_id', 'action'], axis=1)
X_val = train.loc[train.date >= 425].drop(['date', 'ts_id', 'action'], axis=1) 
y_train = train.loc[train.date < 425]['action']
y_val = train.loc[train.date >= 425]['action']

In [9]:
from sklearn.preprocessing import QuantileTransformer

quant_transf = QuantileTransformer(n_quantiles=100, output_distribution='normal')
X_train = quant_transf.fit_transform(X_train)
X_val = quant_transf.transform(X_val)

In [10]:
from sklearn.decomposition import PCA

pca = PCA(n_components=60)
X_train_pca = pca.fit_transform(X_train)
X_val_pca = pca.fit_transform(X_val)

In [11]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier()

lgbm_params = {
    'device': 'gpu',
    'n_jobs': -1,
    'n_estimators': 1000,
    'learning_rate': 0.05,
    'max_depth': -1,
    'num_leaves': 300,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
#     'reg_lambda': 0.3,
#     'reg_alpha': 0.3
}

lgbm.set_params(**lgbm_params) 
lgbm.fit(X_train, y_train)

LGBMClassifier(colsample_bytree=0.8, device='gpu', learning_rate=0.05,
               n_estimators=1000, num_leaves=300, subsample=0.8)

In [12]:
lgbm.score(X_val, y_val)

0.5163929121907025

In [13]:
lgbm_pca = LGBMClassifier()
lgbm_pca.set_params(**lgbm_params)
lgbm_pca.fit(X_train_pca, y_train)

LGBMClassifier(colsample_bytree=0.8, device='gpu', learning_rate=0.05,
               n_estimators=1000, num_leaves=300, subsample=0.8)

In [ ]:
lgbm_pca.score(X_val_pca, y_val)

0.5027149933522373